<a href="https://colab.research.google.com/github/Bainer13/Data_BBVA_Team_1/blob/Espe/Sprint_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Armado del entorno



In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [4]:
ls

sample_data/  spark-3.3.2-bin-hadoop3.tgz


In [5]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [6]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [7]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [8]:
!echo $JAVA_HOME

In [9]:
!pip install -q findspark

In [10]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=65e1ae78994ff9f93e6a8e4adc95eb216485fb77c1204e4eb9a6122611a299e0
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


#Librerias y funcionamiento de spark

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark import SparkContext
from pyspark.sql.functions import row_number, desc

In [12]:
spark = SparkSession.builder.getOrCreate()

In [13]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

#Dataframes. 

In [14]:
df_address = spark.read.parquet('/content/sample_data/Datasets/address_bootcamp.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/sample_data/Datasets/customer_basics_bootcamp.snappy.parquet')
df_emails = spark.read.parquet('/content/sample_data/Datasets/emails_bootcamp.snappy.parquet')
df_phones = spark.read.parquet('/content/sample_data/Datasets/phones_bootcamp.snappy.parquet')

In [15]:
df_address.show()

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+
|customer_id|add

#Extraccion de datos 



### JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_phones.  

In [35]:
df_customerbasics_phones = df_customer_basics.join(df_phones, ['customer_id'], how='left').select(df_customer_basics.customer_id, df_phones.prefix_phone_id, df_phones.phone_area_id, df_phones.cellphone_prefix_id, df_phones.phone_exchange_id, df_phones.phone_line_id, df_phones.phone_id, df_phones.last_change_date)
df_customerbasics_phones.sort(F.asc('customer_id')).show()

+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+
|customer_id|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|     phone_id|last_change_date|
+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+
|   00000442|             54|          297|                 15|              529|         6284| 712455962424|      2022-07-20|
|   00000442|             54|          297|                 15|              472|         9337| 384498667593|      2022-07-20|
|   00001419|             54|           11|                 15|             6679|         3207| 044982237001|      2022-11-01|
|   00001939|             54|           11|                 15|             3870|         0150| 008960137788|      2022-09-21|
|   00002575|           null|         null|               null|             null|         null|         null|  

### JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_email.

In [33]:
df_customerbasics_emails = df_customer_basics.join(df_emails,['customer_id'], how = 'left').select(df_customer_basics.customer_id, df_emails.email_desc, df_emails.last_change_date)
df_customerbasics_emails.show()

+-----------+--------------------+----------------+
|customer_id|          email_desc|last_change_date|
+-----------+--------------------+----------------+
|   07867437|                null|            null|
|   06813266|                null|            null|
|   29604216|                null|            null|
|   28028529|                null|            null|
|   28366580|                null|            null|
|   02746969|                null|            null|
|   23441482|                null|            null|
|   23610540|                null|            null|
|   25935956|                null|            null|
|   25848977|                null|            null|
|   01957163|                null|            null|
|   06338965|                null|            null|
|   29676364|                null|            null|
|   26205399|   DIONISI@YAHOO.COM|      2019-12-27|
|   03933825|DIGITALIZACION@EE...|      1900-01-01|
|   29416245|                null|            null|
|   29497813

### JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_adress.  

In [34]:
df_customerbasics_address = df_customer_basics.join(df_address,['customer_id'], how='left').select(df_customer_basics.customer_id, 'street_name', 'address_outdoor_id','address_indoor_id', 'indoor_number', 'address_town_name', 'address_department_name', 'zipcode_id', df_address.last_change_date)
df_customerbasics_address.show()

+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+
|customer_id|         street_name|address_outdoor_id|address_indoor_id|indoor_number|   address_town_name|address_department_name|zipcode_id|last_change_date|
+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+
|   07867437|ALTE FRANCISCO SE...|           0001971|         PB      |      3      |CIUDAD AUTONOMA B...|   CAPITAL FEDERAL  ...|     01416|      2009-01-26|
|   06813266|EL TORDO         ...|           0000866|                 |             |ITUZAINGO        ...|   ITUZAINGO        ...|     01714|      2009-01-26|
|   29604216|                null|              null|             null|         null|                null|                   null|      null|            null|
|   28028529|AMENEDO          ...|           0

#3 ultimos telefonos

### Opcion A

---



In [37]:
# Ordenar el DataFrame por fecha de contacto en orden descendente
df_phones_new = df_customerbasics_phones.orderBy(F.desc("last_change_date"))

# Definir una ventana de partición por cliente y ordenar por fecha de contacto
w = Window.partitionBy("customer_id").orderBy(desc("last_change_date"))

# Agregar una columna de número de fila para cada contacto de cliente
df_phones_new = df_customerbasics_phones.withColumn("fila", F.row_number().over(w))

# Seleccionar los tres primeros contactos de cada grupo de clientes
df_phones_new = df_phones_new.filter(df_phones_new.fila <= 3)
df_phones_new.show(100)


+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+----+
|customer_id|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|     phone_id|last_change_date|fila|
+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+----+
|   00000442|             54|          297|                 15|              529|         6284| 712455962424|      2022-07-20|   1|
|   00000442|             54|          297|                 15|              472|         9337| 384498667593|      2022-07-20|   2|
|   00001419|             54|           11|                 15|             6679|         3207| 044982237001|      2022-11-01|   1|
|   00001939|             54|           11|                 15|             3870|         0150| 008960137788|      2022-09-21|   1|
|   00002575|           null|         null|               null|             

### Opcion B

In [40]:
# Define the window specification
window_spec_phones = Window.partitionBy("customer_id").orderBy(df_customerbasics_phones["last_change_date"].desc()) 

# Assign a row number to each row within the window
df_with_rn = df_customerbasics_phones.withColumn("rn", F.row_number().over(window_spec_phones))

#creamos un campo por numero de telefono
df_with_rn = df_with_rn.withColumn("phone_1", F.when(df_with_rn.rn == 1, F.col('phone_id')).otherwise(None))
df_with_rn = df_with_rn.withColumn("phone_2", F.when(df_with_rn.rn == 2,F.col('phone_id')).otherwise(None))
df_with_rn = df_with_rn.withColumn("phone_3", F.when(df_with_rn.rn == 3,F.col('phone_id')).otherwise(None))

# Group by customer_id and select the first non-null value for each phone number column
grouped_df = df_with_rn.groupBy("customer_id").agg(F.first("phone_1", ignorenulls=True),F.first("phone_2", ignorenulls=True),F.first("phone_3", ignorenulls=True))

# Rename the columns to phone_1, phone_2, and phone_3
renamed_df = grouped_df.withColumnRenamed("first(phone_1)", "phone_1") \
                      .withColumnRenamed("first(phone_2)", "phone_2") \
                      .withColumnRenamed("first(phone_3)", "phone_3")
# Show the result
renamed_df.show()

+-----------+-------------+------------+-------+
|customer_id|      phone_1|     phone_2|phone_3|
+-----------+-------------+------------+-------+
|   00000442| 712455962424|384498667593|   null|
|   00001419| 044982237001|        null|   null|
|   00001939| 008960137788|        null|   null|
|   00002575|         null|        null|   null|
|   00002707|   6116490163|        null|   null|
|   00002790| 051880599267|  4330463819|   null|
|   00004287|1456900382957|        null|   null|
|   00004724| 820582605809|        null|   null|
|   00007932| 402572158909|        null|   null|
|   00011850| 163848812051|790484486094|   null|
|   00012051| 226398993225|        null|   null|
|   00012611|   0647016054|162848642527|   null|
|   00012991|         null|        null|   null|
|   00013498| 533666791591|        null|   null|
|   00014664| 451978668757|        null|   null|
|   00015497| 975842902003|        null|   null|
|   00051576| 959943195039|        null|   null|
|   00053993| 730405

In [ ]:
filtro = ["customer_id","phone_1", "phone_2", "phone_3"]
ejemplo = renamed_df.select(filtro)

#3 ultimos mails

In [39]:
# Define the window specification
window_spec_emails = Window.partitionBy("customer_id").orderBy(df_customerbasics_emails["last_change_date"].desc()) 

# Assign a row number to each row within the window
df_with_rn_emails = df_customerbasics_emails.withColumn("rn", F.row_number().over(window_spec_emails))

#creamos un campo por email
df_with_rn_emails = df_with_rn_emails.withColumn("email_1", F.when(df_with_rn_emails.rn == 1, F.col('email_desc')).otherwise(None))
df_with_rn_emails = df_with_rn_emails.withColumn("email_2", F.when(df_with_rn_emails.rn == 2,F.col('email_desc')).otherwise(None))
df_with_rn_emails = df_with_rn_emails.withColumn("email_3", F.when(df_with_rn_emails.rn == 3,F.col('email_desc')).otherwise(None))

# Group by customer_id and select the first non-null value for each phone number column
grouped_df_emails = df_with_rn_emails.groupBy("customer_id").agg(F.first("email_1", ignorenulls=True),F.first("email_2", ignorenulls=True),F.first("email_3", ignorenulls=True))

# Rename the columns to email_1, email_2, and email_3
renamed_df_emails = grouped_df_emails.withColumnRenamed("first(email_1)", "email_1") \
                      .withColumnRenamed("first(email_2)", "email_2") \
                      .withColumnRenamed("first(email_3)", "email_3")
# Show the result
renamed_df_emails.show()

+-----------+--------------------+--------------------+-------+
|customer_id|             email_1|             email_2|email_3|
+-----------+--------------------+--------------------+-------+
|   00000442|                null|                null|   null|
|   00001419|  JU_LY1@HOTMAIL.COM|EMILIA.RUBIANES@H...|   null|
|   00001939|                null|                null|   null|
|   00002575|                null|                null|   null|
|   00002707|                null|                null|   null|
|   00002790| NOTIENE@HOIMAIL.COM|DIGITALIZACION@EE...|   null|
|   00004287|                null|                null|   null|
|   00004724|                null|                null|   null|
|   00007932|                null|                null|   null|
|   00011850|                null|                null|   null|
|   00012051|                null|                null|   null|
|   00012611|                null|                null|   null|
|   00012991|                null|      

#3 ultimas direcciones

In [41]:
# Define the window specification
window_spec_address = Window.partitionBy("customer_id").orderBy(df_customerbasics_address["last_change_date"].desc()) 

# Assign a row number to each row within the window
df_with_rn_address = df_customerbasics_address.withColumn("rn", F.row_number().over(window_spec_address))

#creamos un campo por address
df_with_rn_address = df_with_rn_address.withColumn("address_1", F.when(df_with_rn_address.rn == 1, F.col('street_name')).otherwise(None))
df_with_rn_address = df_with_rn_address.withColumn("address_2", F.when(df_with_rn_address.rn == 2,F.col('street_name')).otherwise(None))
df_with_rn_address = df_with_rn_address.withColumn("address_3", F.when(df_with_rn_address.rn == 3,F.col('street_name')).otherwise(None))

# Group by customer_id and select the first non-null value for each phone number column
grouped_df_address = df_with_rn_address.groupBy("customer_id").agg(F.first("address_1", ignorenulls=True),F.first("address_2", ignorenulls=True),F.first("address_3", ignorenulls=True))

# Rename the columns to address_1, address_2, and address_3
renamed_df_address = grouped_df_address.withColumnRenamed("first(address_1)", "address_1") \
                      .withColumnRenamed("first(address_2)", "address_2") \
                      .withColumnRenamed("first(address_3)", "address_3")
# Show the result
renamed_df_address.show()

+-----------+--------------------+---------+---------+
|customer_id|           address_1|address_2|address_3|
+-----------+--------------------+---------+---------+
|   00000442|AGUERO           ...|     null|     null|
|   00001419|ALTE G BROWN     ...|     null|     null|
|   00001939|JOSE HERNANDEZ   ...|     null|     null|
|   00002575|                null|     null|     null|
|   00002707|GUATEMALA        ...|     null|     null|
|   00002790|AV CORDOBA       ...|     null|     null|
|   00004287|CNEL DE MARINA RO...|     null|     null|
|   00004724|PJE MUTUALISMO   ...|     null|     null|
|   00007932|AV FELIX DE OLAZA...|     null|     null|
|   00011850|JUNCAL           ...|     null|     null|
|   00012051|AV GRAL JOSE MARI...|     null|     null|
|   00012611|ALBERTO LARROQUE ...|     null|     null|
|   00012991|                null|     null|     null|
|   00013498|AV PTE H YRIGOYEN...|     null|     null|
|   00014664|AV TRIUNVIRATO   ...|     null|     null|
|   000154

Agregar una nueva columna a los DataFrame de contactos, indicando el contact_type según corresponda (address, email, phone)
PARA: Enriquecer los datos agregando su categoría

QUIERO: Agregar una nueva columna al DataFrame de contactos telefónicos de clientes, resguardando el contacto en formato json contenido en string, con los datos: Phone_type (mobile, landline ), Código país., Código de Área, Número teléfono
PARA: Enriquecer los datos aplicando el formato requerido

QUIERO: Agregar una nueva columna al DataFrame de direcciones de clientes, resguardando el contacto en formato json contenido en string, con los datos: Calle, Número, Piso, Depto, Localidad, Provincia, Código postal
PARA: Enriquecer los datos aplicando el formato requerido

QUIERO: Combinar los DataFrame de contactos telefónicos de clientes, direcciones de clientes y email de clientes en uno solo.
PARA: Enriquecer los datos, combinar distintas fuentes .

QUIERO: Filtrar el DataFrame de contactos y resguardar solo 3 contactos por cliente respetando la prioridad phone, email, address. Resguardar el resultado como una vista temporal.
PARA: Reducir el volumen de datos y trabajar solo con los más actualizados. **